In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pathlib import Path
from typing import Dict, Text
import random
from collections import Counter
import os

In [2]:
path = f"C:/Users/{os.getlogin()}/Documents/Data/Corise"

article_df = pd.read_csv(f"{path}/hmdata/articles.csv.zip")
customer_df = pd.read_csv(f"{path}/hmdata/customers.csv.zip")
train0 = pd.read_csv(f'{path}/hmdata/transactions_train.csv.zip')
transaction_df = train0

In [3]:
transaction_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:
article_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
article_cols = ["article_id", "garment_group_no"]

train1 = transaction_df.merge(article_df[article_cols], on='article_id', how='left')

In [6]:
train1 = train1[train1['t_dat'] <='2019-09-20']


# add 0 in article_id column (string)
train1['article_id'] = train1['article_id'].astype(str)
train1['article_id'] = train1['article_id'].apply(lambda x: x.zfill(10))
train1.head()

,t_dat,customer_id,article_id,price,sales_channel_id,garment_group_no
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2,1017
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2,1017
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2,1003
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2,1023
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2,1023


In [7]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

# Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)
def estimateRecall(cg, purchase):
    return len(intersection(cg,purchase))/len(purchase)

In [8]:
train = train1[train1['t_dat']<='2019-06-20']
test = train1[train1['t_dat'] >='2019-06-20']

In [9]:
c1 = train['customer_id'].to_list()
c2 = test['customer_id'].to_list()

commonUsers = intersection(c1, c2)

print(len(train),len(test),len(c1),len(c2),len(commonUsers))

12241294 4686893 12241294 4686893 423753


In [10]:
#Candidate Generator 1: random products
    
def getKCandidatesRandom(u,k):
    return random.sample(train['article_id'].to_list(),k)

In [11]:
#Candidate Generator 2: top-k popular products based on historic interactions

def getTopKCandidatesPopular(u,k):
    c = Counter(train['article_id'].to_list())
    topPopular = [article for article, count in c.most_common(k)]
    return topPopular

In [12]:
# with the 2 CGs defined, we can now query these CGs for a set of users,
# and compute the recall metrics to see how well they perform.

# NOTE: changed from assignment -> use 50 users instead of first 10 for validation

def runCandidateGeneration(method,k):
    overallRecall = 0
    recallList = []
    i=0
    #userSet = commonUsers[0:10]
    userSet = commonUsers[0:50]
    for u in userSet:
        purchaseList = test[test['customer_id']==u]['article_id'].to_list()
        cg = method(u,k)
        r = estimateRecall(cg,purchaseList)
        overallRecall+=r
        recallList.append(r)
    overallRecall = overallRecall/len(userSet)
    return (overallRecall,recallList)

In [13]:
%%time
# Running metric calculations for the CG 1: random CG, to fetch 100 and 1000 candidates.
r, rlist = runCandidateGeneration(getKCandidatesRandom,k=100)
print("recall for candidate generation at k=100 candidates fetched: ", r)

r, rlist = runCandidateGeneration(getKCandidatesRandom,k=1000)
print("recall for candidate generation at k=1000 candidates fetched: ", r)

recall for candidate generation at k=100 candidates fetched:  0.001
recall for candidate generation at k=1000 candidates fetched:  0.044705385521902306
Wall time: 38.3 s


In [14]:
%%time
# Running metric calculations for CG 2: top popular products CG, to fetch 100 and 1000 candidates.

r, rlist = runCandidateGeneration(getTopKCandidatesPopular,k=100)
print("recall for candidate generation at k=100 candidates fetched: ", r)

r, rlist = runCandidateGeneration(getTopKCandidatesPopular,k=1000)
print("recall for candidate generation at k=1000 candidates fetched: ", r)

recall for candidate generation at k=100 candidates fetched:  0.03313526570048309
recall for candidate generation at k=1000 candidates fetched:  0.1484206409967486
Wall time: 2min 46s


In [15]:
#Candidate Generator 3: 


frequency_df = train.pivot_table(index="customer_id", columns="garment_group_no", values="article_id", aggfunc="count", fill_value=0).unstack().to_frame().rename(columns={0: "frequency"}).reset_index()
frequency_df["rand"] = np.random.randint(1, 10e6, frequency_df.shape[0])


def getTopKCandidatesUserInterest(u,k):
    """Return k random items from the pool of items within a category user u has already bought from. 
    Only the most 5 categories with the most purchased from that user are kept. 
    In case a user has bought less than 5 distinct categories, random ones are selected to reach 5 categories.
    """
    
    num_categories = 5
    
    most_frequent_categories = frequency_df[frequency_df["customer_id"] == u].sort_values(by=["frequency", "rand"], ascending=False)["garment_group_no"].head(num_categories).to_list()
    
    return random.sample(train[train["garment_group_no"].isin(most_frequent_categories)]['article_id'].to_list(),k)    
    

In [16]:
%%time
# Running metric calculations for CG 3: user interests CG, to fetch 100 and 1000 candidates.

r, rlist = runCandidateGeneration(getTopKCandidatesUserInterest,k=100)
print("recall for candidate generation at k=100 candidates fetched: ", r)

r, rlist = runCandidateGeneration(getTopKCandidatesUserInterest,k=1000)
print("recall for candidate generation at k=1000 candidates fetched: ", r)

recall for candidate generation at k=100 candidates fetched:  0.001
recall for candidate generation at k=1000 candidates fetched:  0.050673321279459387
Wall time: 4min 15s


In [17]:
#Candidate Generator 4: 

def getTopKCandidatesUserPrice(u,k):
    """Return k random items from the pool of items with a price at most 12.5% away 
    from median price of items bought by user u"""
    
    z = .125
    
    median_price = train[train["customer_id"] == u]["price"].median()
    low_bound = median_price * (1-z)
    high_bound = median_price * (1+z)
    
    
    return random.sample(train[(train["price"] >= low_bound) & (train["price"] <= high_bound)]['article_id'].to_list(),k) 

In [18]:
%%time
# Running metric calculations for CG 4: user price preference CG, to fetch 100 and 1000 candidates.

r, rlist = runCandidateGeneration(getTopKCandidatesUserPrice,k=100)
print("recall for candidate generation at k=100 candidates fetched: ", r)

r, rlist = runCandidateGeneration(getTopKCandidatesUserPrice,k=1000)
print("recall for candidate generation at k=1000 candidates fetched: ", r)

recall for candidate generation at k=100 candidates fetched:  0.012118171683389073
recall for candidate generation at k=1000 candidates fetched:  0.05241489309396249
Wall time: 1min 50s


In [19]:
# two-tower metrics from initial notebook:
# Recall@100:  0.0013
# Recall@1000: 0.15